In [1]:

import sys
sys.dont_write_bytecode = True
import pandas as pd
%load_ext autoreload
%autoreload 2
import os


In [1]:
import mlflow
mlflow.set_tracking_uri('http://0.0.0.0:5000')

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x72b6016d53f0>>

In [3]:
run_id = 'c7a9943c2290459386db8ae760ff45f5'
dv_uri = f'runs:/{run_id}/dict_vectorizer'
loaded_dv = mlflow.sklearn.load_model(dv_uri)
logged_model = 'runs:/c7a9943c2290459386db8ae760ff45f5/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

2024/08/05 23:48:52 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.14.3, required: mlflow==2.15.0)
 - lz4 (current: uninstalled, required: lz4==4.3.3)
 - psutil (current: 6.0.0, required: psutil==5.9.8)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [4]:
inp

{'isBasicEconomy': True,
 'isRefundable': False,
 'isNonStop': True,
 'totalFare': 168.61,
 'departure_hour': 18,
 'departure_day': 'Friday',
 'departure_month': 6,
 'days_to_departure': 11,
 'DISTANCE': 757}

In [ ]:
dist

In [3]:
inp = df.iloc[1,:].to_dict()

In [5]:
dist = pd.read_csv('../mlops/dataset/dist.csv')

In [8]:
startingAirport = '1G4'
destinationAirport = '1FL'

In [9]:
dist.query('startingAirport == @startingAirport and destinationAirport == @destinationAirport')

,startingAirport,destinationAirport,DISTANCE


In [6]:
transformed_data = loaded_dv.transform(inp)

NameError: name 'inp' is not defined

In [ ]:
loaded_model.predict(transformed_data)

In [ ]:
from mlops.tracking.mlflow import *

In [ ]:
from mlflow import MlflowClient
client = MlflowClient()

In [ ]:
for exp in client.search_experiments():
    client.delete_experiment(exp.experiment_id)

In [ ]:
mlflow.set_tracking_uri('http://0.0.0.0:5000')
mlflow.search_experiments()[0]

In [ ]:
exp.name

In [ ]:
check_registry_if_not_exist(exp)